In [1]:
import pandas as pd 
import numpy as np 
import requests 
import time 
import datetime 
import json
import glob
import os

# API call - Get
r = requests.get("http://localhost:8009/api/point_mass_sim/simulators")
print(r.json())

# Retrieving the SimId 
for dict in r.json():
    dict_simId = dict['simId']
simId = dict_simId

# Grid of gearRatios 
ratio_grid = np.linspace(0, 8, 100).round(2) 

# API call - Post 
sessionId_array = [None] * len(ratio_grid)
i = 0
for gear_ratio in ratio_grid: 
    r = requests.post("http://localhost:8009/api/point_mass_sim", 
    json = {"simId": simId,
    "carId": "1",
    "title": "test",
    "description": "testing",
    "trackId": "track",
    "parameters": {
        "trackModuleList": [{
            "track": "Standard-Endurance-Track-CCW" # This parameter doesn't work properly atm. 
        }],
        "generalModuleList": [{
            "driverMass": 77,
            "carMass": 200,
            "allWheelDrive": False,
            "frontMassDistribution": 0.45,
            "gravity": 9.81,
            "airDensity": 1.225
        }],
        "suspensionModuleList": [{
            "mu": 1.4,
            "tyreRadius": 0.221
        }],
        "brakeModuleList": [{
            "discRadius": 0.075,
            "muPad": 0.45,
            "pistonArea": 0.0013,
            "boreRadius": 0.007,
            "mrPedal": 1.6
        }],
        "gearModuleList": [{
            "gearChangeTime": 0.1,
            "primaryGearRatio": 2.545,
            "finalDriveRatio": gear_ratio, # CHANGING THE FINALDRIVERATIO ONLY!!!
            "driveLineEfficiency": 0.85,
            "gearRatios": [2.307, 1.8, 1.437, 1.15, 1.0]
        }],
        "powerModuleList": [{
            "powerCurve": [
                [3000, 11370, 0.3484],
                [4000, 17510, 0.3484],
                [5000, 28220, 0.3078],
                [6000, 35840, 0.2961],
                [7000, 38970, 0.293],
                [8000, 41970, 0.2919],
                [9000, 42810, 0.3019],
                [10000, 40180, 0.3049],
                [11000, 40020, 0.321],
                [12000, 34860, 0.321]
            ]
        }],
        "aeroModuleList": [{
            "liftCoefficient": -1.5, 
            "dragCoefficient": 0.79
        }]
    }
    }
    )
    # Retrieving the SessionId for current session 
    for dict in r.json():
        dict_sessionId = r.json()[dict]
        break
    sessionId_array[i] = dict_sessionId
    i = i + 1 

# Results
flag = False 
while flag == False: 
    now = datetime.datetime.now() 
    date_time = now.strftime("%H:%M:%S") 
    print(date_time)

    flag_test_pre = glob.glob('/Users/lexy/telemetry-system/service_data/data/1/2/'+str(sessionId_array[-1]))[0]
    os.chdir(flag_test_pre)
    lis_pre = os.listdir('.')
    if(len(lis_pre) == 2): 
        flag_test = glob.glob('/Users/lexy/telemetry-system/service_data/data/1/2/'+str(sessionId_array[-1])+'/*-*')[0]
        os.chdir(flag_test)
        lis = os.listdir('.')
        if(len(lis) == 2): 
            print("flag is True")
            flag = True
            idx = 0 
            cols = ["GearRatio", "lapTime", "fuelConsumption", "maxBrakeForce"]
            results = pd.DataFrame(columns = cols)
            while idx < len(ratio_grid): 
                path_test = glob.glob('/Users/lexy/telemetry-system/service_data/data/1/2/'+str(sessionId_array[idx])+'/')[0]
                os.chdir(path_test)
                lis = os.listdir('.')
                if(len(lis) == 2):
                    path = glob.glob('/Users/lexy/telemetry-system/service_data/data/1/2/'+str(sessionId_array[idx])+'/*-*')[0]+'/metadata.json'
                    f = open(str(path))
                    out = json.load(f)
                    results.loc[idx] = [out['data']['parameters']['finalDriveRatio'],out['data']['results']['lapTime'], out['data']['results']['fuelConsumption'], out['data']['results']['maxBrakeForce']]
                    f.close()
                os.chdir('..')
                idx = idx + 1 
            break
    time.sleep(5)

[{'simId': 'c7f74fc5-5768-4584-870d-ee0727359755', 'name': 'local_simulator'}]
09:40:21
09:40:26
09:40:31
09:40:36
09:40:41
09:40:46
09:40:51
09:40:56
09:41:01
09:41:06
09:41:11
09:41:16
09:41:21
09:41:26
09:41:31
09:41:36
09:41:41
09:41:46
09:41:51
09:41:56
flag is True


## Results / Exporting

Note that at the moment, we need to run this whole program 3 times changing the code in 'Program.cs' line 183 to the specific map each time. 

In [25]:
results

,GearRatio,lapTime,fuelConsumption,maxBrakeForce
0,0.00,4.832026,0.325431,0.0
1,0.89,4.832026,0.325431,0.0
2,1.78,4.832026,0.325431,0.0
3,2.67,4.832026,0.325431,0.0
4,3.56,4.832026,0.325431,0.0
5,4.44,4.832026,0.325431,0.0
6,5.33,4.832026,0.325431,0.0
7,6.22,4.832026,0.325431,0.0
8,7.11,4.832026,0.325431,0.0
9,8.00,4.832026,0.325431,0.0


In [34]:
results # endurance 

# export to csv 
results.to_csv('endurance_output.csv')

,GearRatio,lapTime,fuelConsumption,maxBrakeForce
0,0.00,23.525341,10.923068,104.644468
1,0.08,23.525341,10.923068,104.644468
2,0.16,23.525341,10.923068,104.644468
3,0.24,23.525341,10.923068,104.644468
4,0.32,23.525341,10.923068,104.644468
...,...,...,...,...
95,7.68,25.092369,2.476532,95.129999
96,7.76,25.275458,2.618625,95.129999
97,7.84,25.459244,2.451205,95.129999
98,7.92,25.643503,2.158550,95.129999


In [9]:
print(results) # acceleration (not currently working atm. Raised this issue with Leo )

# export to csv 
results.to_csv('acceleration_output.csv')

,GearRatio,lapTime,fuelConsumption,maxBrakeForce


In [38]:
results # skidpad 

# export to csv 
results.to_csv('skidpad_output.csv')